# Практическая работа по дисциплине "Теория принятия решений"
## Тема: Применение методов линейного и динамического программирования для решения практических задач (по вариантам)
### Вариант 47

**Половников Никита**

**группа 1374**

## Задача 2 
    
        В регионе имеется 5 горно-добывающих предприятий, каждое из которых характеризуется годовой выработкой руды (см. табл. 4).
        В результате геолого-экономического исследования района выделено 5 точек, где могут быть размещены горно-обогатительные предприятия (ГОП), на которых производится преобразование добываемой руды в конечный продукт.
        Существует 3 типа ГОП, отличающихся максимальной мощностью (выражаемой в объеме обрабатываемого в год сырья) и капитальными (единовременными) затратами на строительство (см. табл. 5).
        Известны также предполагаемые затраты на перевозку тонны сырья от горно-добывающих предприятий в потенциальные точки размещения горно-обогатительных предприятий (см. табл. 6). 
        
        1. Найти план строительства ГОП (размещение и тип) в данном районе, минимизирующий затраты в пятилетней перспективе и позволяющий полностью задействовать мощности добывающих предприятий. 
        
        2. Предположим, при используемой технологии обогащения соотношение руды и полезного продукта, получаемого из нее 445 к 1. При какой цене на продукт найденный в п. 1 план принесет прибыль?

**Таблица 4:Годовая выработка горно-добывающих предприятий**

| **Предприятие** | **Годовая выработка, т** |
|:---:|:---:|
| **1** | 500 |
| **2** |800 |
| **3** | 1200 |
| **4** | 600 |
| **5** | 1400 |

**Таблица 5: Типы горно-обогатительных предприятий**

| **Тип** | **Макс. мощность, т/год** | **Кап. затраты, ДЕ** | 
|:---:|:---:|:---:|
| **1** | 1120 | 800 | 
| **2** | 2058 | 1700 | 
| **3** | 4181 | 2700 | 

**Таблица 6: Стоимость перевозок**

| **ГДП \ГОП** | **1** | **2** | **3** | **4** | **5** | 
|:---:|:---:|:---:|:---:|:---:|:---:|
| **1** | 6.2 | 6.9 | 7.0 | 4.7 | 6.0 | 
| **2** | 6.1 | 10.0 | 7.8 | 2.0 | 9.3 | 
| **3** | 2.5 | 5.4 | 6.4 | 4.4 | 3.7 |
| **4** | 7.4 | 3.9 | 4.4 | 8.5 | 4.5 |  
| **5** | 4.5 | 6.1 | 3.8 | 7.7 | 7.9 | 


### Формализация задачи 

*Решаем задачу минимизации затрат предприятия на отрезке 5 лет.*

**Целевая функция:**

#### $ \normalsize c = 5\sum{\sum{p_{ij}x_{ij}}} + \sum{\sum{y_{j}k_{k}} } \rightarrow min$,

где $\normalsize p_{ij}$ - стоимость перевозки из $i$-го гдп  в $j$-ый гоп,

$\normalsize x_{ij}$ - сколько тонн руды перевозится из $i$-го гдп  в $j$-ый гоп,

$\normalsize x_{ij}$ - сколько тонн руды перевозится из $i$-го гдп  в $j$-ый гоп,

$\normalsize y_{j}$ - флаг того, какого типа строить гоп и строить ли вообще (принимает значения 0 и 1),

a $\normalsize k_{k}$ - капитальные затраты на постройку гдп $k$-го типа.


**Целевую функцию запишем в виде:**

$ \normalsize c = 5( p_{11}x_{11} + p_{12}x_{12} + ... + p_{53}x_{53} ) + y_{1}^{1}k_{1} + y_{1}^{2}k_{2} + y_{1}^{3}k_{3} ... + y_{5}^{1}k_{1} + y_{5}^{2}k_{2} + y_{5}^{3}k_{3}\rightarrow min$

***Ограничения:***

$\bullet$ **По годовой выработке ГДП:**

$ x_{11}+ x_{12}+x_{13} \leq 500 $

...

$ x_{51}+ x_{52}+x_{53} \leq 1400 $


$\bullet$ **На количество ГОП:**

$ y_{1}^{1} + y_{1}^{2} + y_{1}^{3} \leq 1 $

...

$ y_{5}^{1} + y_{5}^{2} + y_{5}^{3} \leq 1 $

$\bullet$ **По мощности ГОП:**

$ x_{11}+ x_{21}+x_{31} + x_{41} + x_{51} \leq 1120y_{1}^{1} + 2058y_{1}^{2} + 4181y_{1}^{3} $

...

$ x_{15}+ x_{25}+x_{35} + x_{45} + x_{55} \leq 1120y_{5}^{1} + 2058y_{5}^{2} + 4181y_{5}^{3} $



### Описание задачи с помощью средств выбранной библиотеки

    1. Найти план строительства ГОП (размещение и тип) в данном районе, минимизирующий затраты в пятилетней перспективе и позволяющий полностью задействовать мощности добывающих предприятий. 

In [1]:
# !pip install pulp
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pulp import *

# Создаем задачу линейного программирования
prob = LpProblem("Minimize Cost", LpMinimize)

# Переменные решения
x = LpVariable.dicts("x", range(25), lowBound=0) # Сколько тонн руды перевозить из i-го гдп в j-ый гоп

y = LpVariable.dicts("y", range(15), lowBound=0, upBound=1, cat='Integer') # флаг того, какого тип строить гоп и строить ли вообще (принимает значения 0 и 1).

prob += 5*(6.2*x[0] + 6.9*x[1] + 7.0*x[2] + 4.7*x[3] + 6.0*x[4] + 6.1*x[5] + 10.0*x[6] + 7.8*x[7] + 2.0*x[8] + 9.3*x[9] + 2.5*x[10] + 5.4*x[11] + 6.4*x[12] +  4.4*x[13] + 3.7*x[14] + 7.4*x[15] + 3.9*x[16] + 4.4*x[17] + 8.5*x[18] + 4.5*x[19] + 4.5*x[20] + 6.1*x[21] + 3.8*x[22] + 7.7*x[23] + 7.9*x[24]) + 800*y[0] + 1700*y[1] + 2700*y[2] + 800*y[3] + 1700*y[4] + 2700*y[5] + 800*y[6] + 1700*y[7] + 2700*y[8] + 800*y[9] + 1700*y[10] + 2700*y[11] + 800*y[12] + 1700*y[13] + 2700*y[14], "Total Cost"

# Добавляем ограничения
prob += x[0]+ x[1] + x[2] + x[3] + x[4] == 500, "Огр. годовой добычи на гдп 1" 
prob += x[5]+ x[6] + x[7] + x[8] + x[9] == 800, "Огр. годовой добычи на гдп 2" 
prob += x[10]+ x[11] + x[12] + x[13] + x[14] == 1200, "Огр. годовой добычи на гдп 3" 
prob += x[15]+ x[16] + x[17] + x[18] + x[19] == 600, "Огр. годовой добычи на гдп 4" 
prob += x[20]+ x[21] + x[22] + x[23] + x[24] == 1400, "Огр. годовой добычи на гдп 5"

prob += y[0]+ y[1] + y[2] <= 1, "Огр. на кол-во гоп 1" 
prob += y[3]+ y[4] + y[5] <= 1, "Огр. на кол-во гоп 2" 
prob += y[6]+ y[7] + y[8] <= 1, "Огр. на кол-во гоп 3" 
prob += y[9]+ y[10] + y[11] <= 1, "Огр. на кол-во гоп 4" 
prob += y[12]+ y[13] + y[14] <= 1, "Огр. на кол-во гоп 5" 

prob += x[0]+ x[5] + x[10] + x[15] + x[20] <= 1120*y[0] + 2058*y[1] + 4181*y[2], "Огр. макс. мощности на гоп 1" 
prob += x[1]+ x[6] + x[11] + x[16] + x[21] <= 1120*y[3] + 2058*y[4] + 4181*y[5], "Огр. макс. мощности на гоп 2" 
prob += x[2]+ x[7] + x[12] + x[17] + x[22] <= 1120*y[6] + 2058*y[7] + 4181*y[8], "Огр. макс. мощности на гоп 3" 
prob += x[3]+ x[8] + x[13] + x[18] + x[23] <= 1120*y[9] + 2058*y[10] + 4181*y[11], "Огр. макс. мощности на гоп 4" 
prob += x[4]+ x[9] + x[14] + x[19] + x[24] <= 1120*y[12] + 2058*y[13] + 4181*y[14], "Огр. макс. мощности на гоп 5"


In [3]:
prob.solve()

# Выводим результат
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", value(prob.objective))


Status: Optimal
x_0 = 0.0
x_1 = 0.0
x_10 = 1120.0
x_11 = 0.0
x_12 = 0.0
x_13 = 80.0
x_14 = 0.0
x_15 = 0.0
x_16 = 600.0
x_17 = 0.0
x_18 = 0.0
x_19 = 0.0
x_2 = 0.0
x_20 = 0.0
x_21 = 0.0
x_22 = 1400.0
x_23 = 0.0
x_24 = 0.0
x_3 = 500.0
x_4 = 0.0
x_5 = 0.0
x_6 = 0.0
x_7 = 0.0
x_8 = 800.0
x_9 = 0.0
y_0 = 1.0
y_1 = 0.0
y_10 = 1.0
y_11 = 0.0
y_12 = 0.0
y_13 = 0.0
y_14 = 0.0
y_2 = 0.0
y_3 = 1.0
y_4 = 0.0
y_5 = 0.0
y_6 = 0.0
y_7 = 1.0
y_8 = 0.0
y_9 = 0.0
Total Cost = 78810.0


**План строительства ГОП (размещение и тип) в данном районе, минимизирующий затраты в пятилетней перспективе и позволяющий полностью задействовать мощности добывающих предприятий:**

- **ГОП1 - 1 тип** (переменная y_0)
- **ГОП2 - 1 тип** (переменная y_3)
- **ГОП3 - 2 тип** (переменная y_7) 
- **ГОП4 - 2 тип** (переменная y_10)
- **Строить ГОП5 необходимости нет.** (переменные y_12, y_13, y_14 равны 0)

Стоимость = **78810.0** ДЕ

    2. Предположим, при используемой технологии обогащения соотношение руды и полезного продукта, получаемого из нее 445 к 1. При какой цене на продукт найденный в п. 1 план принесет прибыль?

**Общее количество ежегожно добываемой руды** = ***4500*** тонн

- **Себестоимость 1 тонны полезного продукта = $\frac{Стоимость}{Кол-во \ полезного \ продукта \ в \ руде}$**

- **Кол-во полезного продукта в руде = $\frac{5*4500}{445} = 50.56 \ т.$**

- **Себестоимость 1 тонны полезного продукта = $\frac{78810}{50.56} = 1558.68 \ ДЕ$**

**При цене одной тонны большей, чем себестоимость полезного продукта план принесет прибыль.**


## Вывод:

 - В ходе решения задали линейного програмиирования был найден оптимальный план добычи, минимизирующий затраты на отрезке 5 лет. 
 - Найдена цена продукта, при которой план принесет прибыль.